# A Tutorial for GPU4PySCF

In [ ]:
import numpy as np
import cupy
import pyscf

---

## PySCF Input
* Geometry
* Basis sets
* Pseudo potentials
* Total charge, spin multiplicities, symmetry, etc.
* Methods and corresponding attributes
* Verbose level and other global parameters

### Input for Molecules
* Using the general initialization method `pyscf.M` to instantiate a `Mole` instance.
* Alternatively, molecule instances can be explicitly initialized with the `pyscf.gto.Mole` class
* More examples and documents can be found in https://pyscf.org/user/gto.html and https://github.com/pyscf/pyscf/tree/master/examples/gto

In [ ]:
mol = pyscf.M(
    atom = 'O 0 0 0; O 0 0 1.2',
    basis = 'cc-pvdz',
    spin = 2, # n_alpha - n_beta, corresponding to triplet
    verbose = 4
)

### Input for Materials
* `Cell` instance can be created using the general `pyscf.M` method, with lattice parameters`a`.
* Alternatively, cell in a crystal can be explicitly initialized with the `pyscf.pbc.gto.Cell` class
* More examples and documents can be found in https://pyscf.org/user/pbcgto.html and https://github.com/pyscf/pyscf/tree/master/examples/pbc

In [ ]:
cell = pyscf.M(
    a = '''
    0.0 3.6 3.6
    3.6 0.0 3.6
    3.6 3.6 0.0
    ''',
    atom = '''
    C 0 0 0
    C 1.8 1.8 1.8''',
    basis = 'gth-dzvp',
    pseudo = 'gth-pbe',
)

### Geometry input format
PySCF supports various methods to specify geometry (atomic positions):
* Cartesian coordinates defined in a string, atoms are separated by ';' or '\n'
* Z-matrix format in a string
* Internal format represented by a nested Python list
* String and Python list mixed inputs
* Geometry file (xyz format). Lattice parameter cannot be read from the geometry file.
 
Corresponding example https://github.com/pyscf/pyscf/blob/master/examples/gto/01-input_geometry.py

In [ ]:
mol = pyscf.M(
    atom = 'Vitamin_C.xyz', # Read geometry from xyz file
    basis = 'cc-pvdz',
)

In [ ]:
## Lattice parameters can be defined as a Numpy array
cell = pyscf.M(
    a = np.diag([30, 30, 30]),
    atom = 'H 0 0 0; H 0 0 0.74',
    basis = 'gth-dzvp',
    pseudo = 'gth-pbe',
)

### Basis set input
* A universal basis set, specified by name, for all all elements.
* Different basis set for different elements or atoms.
* Strings to input custom basis set.
* Specifying a local file that stores basis set.
* Basis uncontraction, basis truncation.

Supported basis sets:
* All basis sets provided by the basis set exchange project (https://www.basissetexchange.org/), including def2 family, Pople basis, Dunning basis (cc-pv*z) etc.
* GTH basis family particularly for PBC calculations. 

In [ ]:
mol = pyscf.M(
    atom = '''
O    0.000000    0.000000    0.117790
H    0.000000    0.755453   -0.471161
H    0.000000   -0.755453   -0.471161''',
    basis = {'O': 'cc-pvdz', 'H': 'sto-3g'})

### Pseudopotential (PP) and Effective Core Potential (ECP)
* PP and ECP can be provided in the same way as inputting basis sets: by specifying a universal name, using element-specific names, or by supplying local files containing the pseudopotential data.
* GTH pseudopotentials can be used for molecular calculations, while ECPs are applicable to periodic boundary condition (PBC) calculations.

In [ ]:
mol = pyscf.M(
    atom='Fe 0. 0. 0.; O  0.  0.  1.',
    basis={'Fe': 'GTH-DZVP-MOLOPT-SR', 'O': 'cc-pvdz'},
    pseudo = {'Fe':'GTH-pade'})

### Total charge, spin multiplicities, symmetry, etc.
* Setting `mol.charge` for the total charge of molecule.
* Setting `mol.spin` or `cell.spin` to integer 0, 1, 2, ... for the open-shell systems.
  _Note_: the `spin` attributes indicates the unpaired electrons = $N_{\alpha} - N_{\beta}$
* Enabling `mol.symmetry` or `cell.symmetry` will enable point-group symmetry for molecular calculations and space-group symmetry for PBC calculations.

In [ ]:
mol = pyscf.M(
    atom='Mn 0. 0. 0.; O  0.  0.  1.',
    basis='cc-pvdz',
    spin=5, # 5 un-paired electrons for S=5/2, sextet 
    symmetry=True, # Cylindrical symmetry Cinfv will be identified and utilized in the calculation
)

### Verbose and global control parameters

In [ ]:
mol.verbose = 3 # The default setting, only print the final results
mol.verbose = 4 # More information of the computation configurations and important messages during the computation.
mol.verbose = 5 # More messages. Timing will be printed at this level.
mol.verbose = 6 # Many debugging details.

---

## Electronic Structure Computation Methods
* Python instance for quantum chemistry models
* HF and DFT
* Post-HF and post-DFT
* Properties
* Analyze results
* Offload to GPU

### Python instance for quantum chemistry models

Specific quantum chemistry methods are implemented as Python classes (such as RHF, TDRKS, CCSD). These classes can be imported and instantiated.

In [ ]:
from pyscf.dft.rks import RKS # restricted Kohn-Sham method
mf = RKS(mol, xc='b3lyp').run()


Methods can also be instantiated via shortcut methods provided by the Mole and Cell classes.

In [ ]:
mf = mol.RKS(xc='b3lyp')
mc = mol.CASSCF(4, 4)


A chain of calls can be used to apply subsequent methods, such as running a post-HF method based on an HF computation, or computing gradients for an energy evaluation method. The final `.run()` method must be executed to ensure the computation is executed. The intermediate `.run()` methods can be skipped. 

In [ ]:
mol.RKS(xc='pbe0').run().TDA().run()
mol.HF().run().CCSD().run().Gradients().run()


Certain methods can have parameters (or options) to adjust the calculations. For example, the XC functional, frozen orbitals, or the number of roots/excited states to compute. A computation can be configured by setting the corresponding attributes of the method instance.

In [ ]:
td = mol.RKS(xc='pbe0').run().TDA()
td.nstates = 10 # Compute more excited states than the default 3 states.


The `.analyze()` method is por DFT calculations and TDDFT calculations. It summarizes key results, such as the total energy, one-electron and two-electron contributions, orbital energies, and Mulliken charges and electron populations.

In [ ]:
mf.analyze()
td.analyze()

PySCF computations can be accelerated using GPU4PySCF. The performance gains are especially pronounced for DFT, DFT excited states, and electromagnetic properties at the DFT level.

To integrate GPU4PySCF into PySCF code:
* `to_gpu()` converts a PySCF instance into a GPU4PySCF instance.
* `to_cpu()` converts a GPU4PySCF instance back into a PySCF instance.

In [ ]:
mf_on_gpu = mol.RKS(xc='wb97mv').density_fit().to_gpu().run()
td_on_cpu = mf.TDA().to_cpu().run()

Note: by design, the `to_gpu()` and `to_cpu()` method can be performed at any place before calling `run()`. However, some conversion code were not implemented in either the PySCF package or GPU4PySCF package. It's recommended to place the conversion calls right after the mean-field object (before any subsequent operations)

### Examples

Run a standard DFT computation using default algorithms and default settings.

In [ ]:
mol.KS(xc='pbe0').run() # on CPUs
mol.KS(xc='pbe0').to_gpu().run() # on GPUs

For crystal systems with periodic boundary conditions, Gamma-point and k-mesh sampling calculations can be instantiated as

In [ ]:
cell.KS(xc='pbe').to_gpu().run()
cell.KKS(xc='pbe', kpts=cell.make_kpts([2,2,2])).to_gpu().run()

Depending on the system size, DFT functionals, required accuracy, band structures, etc., default algorithm might not be the optimal choice.

Use density fitting approximation to improve the performance for small and medium size molecules (e.g. Natom < 100). However, for large-size molecules (e.g. Natom > 200), the default algorithm is more efficient.

In [ ]:
mol.KS(xc='pbe0').density_fit().run()
mol.KS(xc='pbe0').density_fit().to_gpu().run()

Set a different auxiliary basis set than the default one. The default auxiliary basis sets are configured to provide accurate integral for both J and K matrices (JK-fit), typically a large basis set. This basis can be replaced by a small J-fit basis when running a local or semi-local DFT functional.

In [ ]:
mol.KS(xc='pbe').density_fit(auxbasis='def2-universal-jfit').to_gpu().run()

For PBC systems, the more efficient multigrid algorithm can be enabled for local or semi-local functionals

In [ ]:
cell.KS(xc='pbe').to_gpu().multigrid_numint().run()

Solvation effects can be performed in the DFT computation. This correction will be automatically applied for subsequent methods.

In [ ]:
mol.KS(xc='b3lyp').to_gpu().PCM().run()
mol.KS(xc='b3lyp').to_gpu().PCM().run().TDA().run()

Relativistic corrections can be performed. Additionally, relativistic effects can be applied along with the solvation model, and they can be specified in arbitrary order.

In [ ]:
mol.KS(xc='b3lyp').to_gpu().x2c().run()
mol.KS(xc='b3lyp').to_gpu().x2c().PCM().run()

For challenging systems with slow SCF convergence, second order convergence algorithm can be applied.

In [ ]:
mol.KS(xc='b3lyp').to_gpu().soscf().run()

In DFT+U calculations, the effective U values and their correspondng orbital sites need to be specified.

In [ ]:
pyscf.M(
    atom='Fe 0. 0. 0.; O 1.8 0 0',
    basis='def2-svp',
    verbose=4
).RKSpU(mol, xc='svwn', U_idx=["Fe 3d"], U_val=[2.0]).to_gpu().run()

For gapless systems, smearing can be performed to improve convergence

In [ ]:
mol.KS(xc='pbe').to_gpu().smearing().run()

Explicitly call an unrestricted Kohn-Sham solver (UKS) or a restrict Kohn-Sham solver, regardless of the spin multiplicity. UKS can be performed for closed-shell systems to obtain spin-symmetry broken results. RKS can be performed for open-shell systems, in the ROKS framework.

In [ ]:
mol.UKS(xc='pbe0').to_gpu().run()
mol.RKS(xc='pbe0').to_gpu().run()

Execute TDDFT excited states computation on top of a KS computation, then view the oscillation strength and natural transition analysis

In [ ]:
mol.KS(xc='pbe0').run().TDA(nstates=5).run().analyze()

Post-HF methods

In [ ]:
mol.HF().run().CCSD().run()
mol.HF().run().CASSCF(4, 4).run()

Nuclear Gradients can be computed after energy minimization

In [ ]:
mol.KS(xc='pbe0').to_gpu().density_fit().run().Gradients().run()
mol.KS(xc='pbe0').run().TDA().run().Gradients().run()

## PySCF for Fast Prototype Development
### PySCF and GPU4PySCF APIs
* Side-effect-free functions
  - Most functions in both PySCF and GPU4PySCF are designed to be free of side effects. They can be executed repeatedly and will always produce the same output, regardless of how many times or when they are called.

* Compatibility between PySCF and GPU4PySCF APIs
  - Most GPU4PySCF functions return CuPy arrays, while PySCF only operates NumPy arrays. Although NumPy and CuPy arrays have similar data structures, they cannot be mixed within the same calculation.
  - Some GPU4PySCF functions introduce additional parameters to enable GPU-specific optimizations.

* Calling GPU4PySCF functions from PySCF code
  - GPU4PySCF function signatures are highly similar to their PySCF counterparts. Functions such as `get_jk` (for computing J and K matrices) and `eval_ao` (for evaluating atomic orbitals on grids) can be substituted directly when these APIs are used.
  - Use cupy.asnumpy() to convert the output of GPU4PySCF functions back to NumPy arrays when needed.


### Example: Evaluating potential on grids
Given an orbital $\psi = \sum_s  C_s \phi_s(\mathbf{r})$ and a density matrix $\gamma$, we aim to compute its exchange potential on real-space grids $[\hat{K}\psi](\mathbf{r})$.

\begin{align}
[\hat{K}\psi](\mathbf{r})
&= \sum_{qr} \gamma_{qr}\int \phi_q(\mathbf{r}) \frac{1}{|\mathbf{r}-\mathbf{r'}|} \phi_r(\mathbf{r'})^* \psi(\mathbf{r'}) d\mathbf{r'} \\
&= \sum_{qr}\gamma_{qr} \sum_s V_{rs}(\mathbf{r}) C_s
\end{align}

where $V_{rs}(\mathbf{r})$ is the Coulomb potential generated by the orbital product $\phi_r^*(\mathbf{r}')\phi_s(\mathbf{r}')$.
\begin{align}
V_{rs}(\mathbf{r})
&= \int \frac{1}{|\mathbf{r}-\mathbf{r'}|} \phi_r(\mathbf{r'})^* \phi_s(\mathbf{r'}) d\mathbf{r'} \\
\end{align}
This potential can be evaluated on reciprocal-space grids ($\mathbf{G}$) and then transformed to real space
\begin{align}
V_{rs}(\mathbf{G}) &= \frac{4\pi}{G^2} \mathrm{FT}[\phi_r^*(\mathbf{r}) \phi_s(\mathbf{r})] \\
V_{rs}(\mathbf{r}) &= \mathrm{IFT}[V(\mathbf{G})]
\end{align}

The evaluatsion of $[\hat{K}\psi](\mathbf{r})$ consists of five steps:
1. Evaluate orbitals $\phi_r(\mathbf{r})$ and $\psi(\mathbf{r})$ on real-space grids;
2. Fourier transform their products:
\begin{equation}
\rho_r(\mathbf{G}) = \mathrm{FT}[\phi_r^*(\mathbf{r})\psi(\mathbf{r})]
\end{equation}
3. Apply Coulomb kernel $\frac{4\pi}{G^2}$ in reciprocal space
\begin{equation}
V_r(\mathbf{G}) = \frac{4\pi}{G^2}\rho_r(\mathbf{G})
\end{equation}
4. Inverse Fourier transform to obtain the Coulomb potential on real-space grids
\begin{equation}
V_r(\mathbf{r}) = \mathrm{IFT}[V_r(\mathbf{G})]
\end{equation}
5. Contract with the density matrix and the orbital values on grids
\begin{equation}
[\hat{K}\psi](\mathbf{r}) = \sum_{qr} \gamma_{qr} \phi_q(\mathbf{r}) V_r(\mathbf{r})
\end{equation}

#### PySCF implementation

In [ ]:
def exchange_potential(cell, C_psi, density_matrix, mesh):
    from pyscf.pbc.tools import fft, ifft
    from pyscf.pbc.dft.numint import eval_ao
    # Set up uniform grids
    a = cell.lattice_vectors()
    grids = cell.get_uniform_grids(mesh)
    
    # 1. orbitals on real space grids
    aoR = eval_ao(cell, grids)
    psi = np.einsum('gs,s->g', aoR, C_psi)

    # 2. Fourier transform orbital products
    orbital_product = np.einsum('gr,g->gr')
    rho_G = fft(orbital_product.T, mesh).T

    # 3. Coulomb potential in reciprocal space
    Gv = cell.get_Gv(mesh)
    G2 = np.einsum('Gx,Gx->G', grids, grids)
    V_G = np.einsum('g,gr->gr', 4*np.pi/G2, rho_G) / cell.vol

    # 4. Inverse Fourier transform
    V_r = ifft(V_G.T, mesh).T

    # 5. contract with orbitals and density matrices
    K = np.einsum('qr,q,gr->g', density_matrix, aoR, V_r)
    return K

#### Acceleration using GPU4PySCF

In [ ]:
def exchange_potential(cell, C_psi, density_matrix, mesh):
    from gpu4pyscf.pbc.tools import fft, ifft
    from gpu4pyscf.pbc.dft.numint import eval_ao
    # Set up uniform grids
    a = cell.lattice_vectors()
    grids = cell.get_uniform_grids(mesh)
    
    # 1. orbitals on real space grids
    aoR = eval_ao(cell, grids)
    psi = cupy.einsum('gs,s->g', aoR, C_psi)

    # 2. Fourier transform orbital products
    orbital_product = cupy.einsum('gr,g->gr')
    rho_G = fft(orbital_product.T, mesh).T

    # 3. Coulomb potential in reciprocal space
    Gv = cell.get_Gv(mesh)
    G2 = cupy.einsum('Gx,Gx->G', grids, grids)
    V_G = cupy.einsum('g,gr->gr', 4*np.pi/G2, rho_G) / cell.vol

    # 4. Inverse Fourier transform
    V_r = ifft(V_G.T, mesh).T

    # 5. contract with orbitals and density matrices
    K = cupy.einsum('qr,q,gr->g', density_matrix, aoR, V_r)
    return K

---

## Interactions with Other Packages

### Geometry Optimization with geomeTRIC or ASE

Molecular geometry optimization and crystal lattice optimization can be performed using the geomeTRIC and ASE packages. To siplify the coding for geometry optimization workflow, PySCF provides a wrapper interface. For methods implemented in both PySCF and GPU4PySCF, the `.optimizer()` method of the Gradients class can be used to set up and run geometry optimization jobs.

In [ ]:
mol.RKS(xc='wb97mv').to_gpu().Gradients().optmizer().run()
# Optimize both lattice and atomic positions
cell.KRKS(xc='pbe', kpts=cell.make_kpts([2]*3)).to_gpu().Gradients().optimizer().run()

If a custom model is implemented that can provide the energy and its derivatives with respect to nuclear coordinates, the `as_pyscf_method` wrapper can be used to convert the model into a PySCF-compatible data structure. Using this wrapper, geometry optimization can be performed in the same way as with any other PySCF method.

In [ ]:
from pyscf.geomopt.addons import as_pyscf_method
as_pyscf_method(mol, energy_and_grad_function).Gradients().optimizer().run()

### ASE interface
PySCF offers the `pyscf_ase` interface, with features including

#### Import crystal structure from ASE
In PBC calculations, manually entering atomic positions and lattice parameters can be inconvenient. Crystal structures can be constructed quickly using the ASE lattice module, and PySCF offers an interface to directly import these ASE-generated structures.

In [ ]:
from ase.build import bulk
from pyscf.pbc.tools.pyscf_ase import cell_from_ase
atoms = bulk('C', 'diamond', a=3.5668)
cell = cell_from_ase(atoms)

#### Converting Mole and Cell objects to ASE Atoms objects

In [ ]:
from pyscf.pbc.tools.pyscf_ase import pyscf_to_ase_atoms
atoms = pyscf_to_ase_atoms(cell)

#### ASE Calculator
ASE provides a general Calculator class to interface with external simulation packages. The `PySCF` calculator supports all methods implemented in PySCF, including the fictitious methods created by `as_pyscf_method()`.

In [ ]:
from pyscf.pbc.tools.pyscf_ase import PySCF
atoms = pyscf_to_ase_atoms(cell)
atoms.calc = ase.PySCF(method=mol.RKS(xc='pbe').density_fit().PCM())

### Visualization
#### Cube format
The pyscf.tools.cubegen module can be used to export potentials or densities on real-space grids in cube format, which can then be visualized using tools such as VMD or Jmol.

In [ ]:
from pyscf.tools import cubegen
mf = cell.RKS(xc='pbe').run()
# electron density
cubegen.density(mol, 'density.cube', mf.make_rdm1())

# The exchange potential for HOMO
nx, ny, nz = [80, 80, 80]
K = exchange_potential(mol, mf.mo_coeff[:,mf.mo_occ>0][:,-1], mf.make_rdm1(), (nx, ny, nz))
cubegen.Cube(cell, nx, ny, nz).write(K, 'HFX_HOMO.cube')

#### Molden format
The `pyscf.tools.molden` module can be used to export orbitals in the Molden format

In [ ]:
from pyscf.tools import molden
mf = mol.RKS(xc='pbe').run()
molden.from_scf(mf, 'scf_results.molden')

### Orbital format conversion

Orbital coefficients in PySCF and in other quantum chemistry packages often follow different conventions for orbital ordering and normalization. Results from external programs, such as ORCA or Gaussian, can be imported into PySCF by converting orbital coefficients using orbkit (https://github.com/orbkit/orbkit) or cclib (https://github.com/cclib/cclib).

---

## High Throughput Computation and Cloud Deployment

Problems to consider in a high throughput task
* How to launch computations?
    * EC2 instances
    * ECS containers
    * GPU instances
    * AWS Batch or k8s cluster

* How to transfer and store data?
    * JSON in RESTful structure
    * Persistent storage within the docker image
    * Hosted in a Git repo which can be accessible by the remote service
    * Database or message queue service for input and output
    * Object storage (S3) for input and output

* How to scale?
    * Auto-scaling group (ASG)
    * AWS batch
    * One-shot computation via serverless or a daemon for repeated requests

### DFT Computation Service Hosted on Volcano Cloud
`pip install volcengine-qcclient`

In [ ]:
from volcengine_qcclient import QcClient
task_config = '''
basis: def2-tzvp
xc: pbe0
with_df: True
save_mo: True
with_grad: True
'''
client = QcClient()
job = client.submit(task_type='sp', task_config=task_config, molecules='/path/to/geometry/folder/')
job.download_outputs()